# Unified Pipeline Demo

Bu notebook, `risk_pipeline` paketinin tek Config tabanlý pipeline'ýný sentetik veri üzerinde gösterir.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from risk_pipeline.core.config import Config
from risk_pipeline.pipeline import RiskModelPipeline

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output
    WIDGETS_AVAILABLE = True
except ImportError:
    WIDGETS_AVAILABLE = False
    print('ipywidgets bulunamadý; varsayýlan ayarlar kullanýlacak.')

DEFAULT_ALGORITHMS = [
    'logistic',
    'gam',
    'catboost',
    'lightgbm',
    'xgboost',
    'randomforest',
    'extratrees',
]

def build_default_config():
    return Config(
        target_column='target',
        id_column='application_id',
        time_column='application_date',
        algorithms=DEFAULT_ALGORITHMS,
        enable_scoring=True,
        enable_dual=True,
        use_optuna=False,
        optimize_risk_bands=True,
        calculate_shap=False,
        report_components=[],
    )

user_config = build_default_config()

if WIDGETS_AVAILABLE:
    alg_widget = widgets.SelectMultiple(
        options=DEFAULT_ALGORITHMS,
        value=tuple(DEFAULT_ALGORITHMS),
        description='Algoritmalar',
        rows=len(DEFAULT_ALGORITHMS),
    )
    scoring_widget = widgets.Checkbox(value=True, description='Skorlama')
    dual_widget = widgets.Checkbox(value=True, description='Dual (WOE+RAW)')
    optuna_widget = widgets.Checkbox(value=False, description='Optuna HPO')
    riskband_widget = widgets.Checkbox(value=True, description='Risk band opt.')

    output = widgets.Output()

    def apply_settings(_):
        global user_config
        user_config = build_default_config()
        user_config.algorithms = list(alg_widget.value) or DEFAULT_ALGORITHMS
        user_config.enable_scoring = scoring_widget.value
        user_config.enable_dual = dual_widget.value
        user_config.use_optuna = optuna_widget.value
        user_config.optimize_risk_bands = riskband_widget.value
        with output:
            clear_output()
            print('Ayarlar güncellendi. Seçilen algoritmalar:', user_config.algorithms)

    apply_button = widgets.Button(description='Ayarlarý Uygula', button_style='success', icon='check')
    apply_button.on_click(apply_settings)

    ui = widgets.VBox([
        widgets.HTML('<b>Pipeline Konfigürasyon Sihirbazý</b>'),
        widgets.HBox([
            alg_widget,
            widgets.VBox([scoring_widget, dual_widget, optuna_widget, riskband_widget, apply_button]),
        ]),
        output,
    ])
    display(ui)
else:
    print('ipywidgets bulunamadý; varsayýlan ayarlar kullanýlacak.')

## Sentetik Eðitim & Kalibrasyon Verisi
Örnek veri setleri oluþturarak pipeline'ý çalýþtýracaðýz.

In [ ]:
def make_demo_datasets(n_train=2000, n_calibration=600, seed=42):
    rng = np.random.default_rng(seed)

    def _build(n):
        df = pd.DataFrame({
            'application_id': np.arange(n),
            'application_date': pd.date_range('2020-01-01', periods=n, freq='D'),
            'age': rng.integers(18, 70, size=n),
            'income': rng.normal(5000, 1500, size=n),
            'credit_limit': rng.normal(10000, 2500, size=n),
            'delinquency_ever': rng.binomial(1, 0.3, size=n),
            'open_accounts': rng.integers(1, 6, size=n),
        })
        score = (
            0.03 * df['age']
            - 0.0005 * df['income']
            - 0.0008 * df['credit_limit']
            + 0.25 * df['delinquency_ever']
            + 0.12 * df['open_accounts']
        )
        probability = 1 / (1 + np.exp(-score / 12))
        df['target'] = rng.binomial(1, probability)
        return df

    train_df = _build(n_train)
    cal_df = _build(n_calibration)
    return train_df, cal_df

train_df, calibration_df = make_demo_datasets()
print(train_df.head())
print('Train default rate:', train_df['target'].mean().round(3))
print('Calibration default rate:', calibration_df['target'].mean().round(3))

## Pipeline Eðitimi

In [ ]:
pipeline = RiskModelPipeline(user_config)
results = pipeline.fit(train_df, calibration_df=calibration_df)

print(f"Best model: {results['best_model']}")
print(f"Seçilen özellik sayýsý (WOE): {len(results['selected_features'].get('woe', []))}")
print(f"Seçilen özellik sayýsý (RAW): {len(results['selected_features'].get('raw', []))}")

## Skorlama Örneði

In [ ]:
if user_config.enable_scoring:
    scores = pipeline.score(train_df.head(5), return_calibrated=True)
    print('Örnek skorlar:', scores)
else:
    print('Scoring devre dýþý. enable_scoring=True yaparak aktifleþtirebilirsiniz.')

## Risk Band Analizi

In [ ]:
if user_config.optimize_risk_bands and pipeline.risk_bands:
    band_stats = pipeline.risk_bands.get('band_stats')
    if band_stats is not None:
        display(band_stats.head())
    else:
        print('Risk band istatistiði bulunamadý')
else:
    print('Risk band optimizasyonu kapalý veya sonuç üretilmedi.')

## Kaydetme & Yükleme

In [ ]:
import tempfile
from pathlib import Path as _Path

checkpoint_dir = _Path(tempfile.gettempdir()) / 'risk_pipeline_demo'
checkpoint_dir.mkdir(exist_ok=True)

model_path = checkpoint_dir / 'trained_pipeline.pkl'
pipeline.save_pipeline(str(model_path))

loaded_pipeline = RiskModelPipeline.load_pipeline(str(model_path))
print('Kaydedilen pipeline yüklendi; best model:', loaded_pipeline.best_model)

## Sonraki Adýmlar
- `train_df` ve `calibration_df` verilerini kendi dataset’inizle deðiþtirin.
- `user_config` ayarlarýný deðiþtirip farklý kombinasyonlar deneyin.
- `python -m pytest tests/integration` komutuyla entegrasyon testlerini çalýþtýrýn.